## Enoncé

Peut-on établir un lien entre la densité de médecins par spécialité et par territoire et la pratique du dépassement d'honoraires ? Est-ce dans les territoires où la densité est la plus forte que les médecins pratiquent le moins les dépassements d'honoraires ? Est-ce que la densité de certains médecins / praticiens est corrélée à la densité de population pour certaines classes d'âges (bébé/pédiatre, personnes âgées / infirmiers etc...) ?

C'est un sujet issu du Hackathon ["données de santé"](https://www.etalab.gouv.fr/retour-sur-le-premier-hackathon-donnees-de-sante) de Etalab (la page contient des liens vers différents jeux de données utiles, ainsi que d'autres exemples de questions que vous pouvez préférer aborder). Vous pouvez compléter ça avec des données de l'INSEE et des données sur la [démographie des médecins](http://www.data.drees.sante.gouv.fr/ReportFolders/reportFolders.aspx?IF_ActivePath=P,490,497,514).

Vous pourrez trouver un jeu de données listant le taux moyen (je pense) de dépassements d'honoraires par spécialité des médecins et par département sur le lien suivant.
https://www.ameli.fr/l-assurance-maladie/statistiques-et-publications/donnees-statistiques/professionnels-de-sante-liberaux/honoraires/honoraires-totaux-et-moyens.php


Un nouveau dataset pour l'exercice kit datascience sur l'estimation de la population en France. Il y a des informations sur les tranches d'âge
https://www.insee.fr/fr/statistiques/1893198

## Dataset utilisés
wiki hackathon : https://github.com/SGMAP-AGD/DAMIR/wiki/hackathon

démographie médecin : http://www.data.drees.sante.gouv.fr/ReportFolders/reportFolders.aspx?IF_ActivePath=P,490,497,514

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [ ]:
df_honoraires = pd.read_excel("./data/Honoraires.xls", sheet_name = "Spécialistes")
df_population = pd.read_excel("./data/estim-pop-dep-sexe-gca-1975-2019.xls", sheet_name ='2019', header =[3,4])

In [ ]:
df_population = pd.read_excel("./data/estim-pop-dep-sexe-gca-1975-2019.xls", sheet_name = 4, skiprows=[0, 1, 2, 3])

#drop NA values
df_population.dropna(inplace = True)

#rename columns 
columns = df_population.columns

ensemble_columns = list(map(lambda x : x, columns[2:8])) #[:-2] to remove .1 and .2 at the end of the column name.
men_columns = list(map(lambda x : x[:-2] + " (H)", columns[8:14]))
women_columns = list(map(lambda x : x[:-2] + " (F)", columns[14:]))

revised_columns = ['Département','Département_name'] + ensemble_columns + men_columns + women_columns

df_population.columns = revised_columns

#df_population["Num_dep"] = df_population["Num_Dep"].astype(str)

In [ ]:
sheet_names = ["Spécialistes", "Généralistes et MEP", "Dentistes et ODF", "Sages-femmes", "Auxiliaires médicaux"]

In [ ]:
def fees_data(sheet_name):
    
    df_fees = pd.read_excel("./data/Honoraires.xls", sheet_name=sheet_name, na_values="nc")
    
    #retrieve type of doctor 
    type_doctor = df_fees.columns[0]
    
    df_fees.columns = ["Spécialité", "Département", "Effectifs", "Honoraires sans dépassement", "Dépassements",
                       "Frais de déplacement", "Total des honoraires"]
    
    
    #remove rows contraining words TOTAL within column Department
    df_fees = df_fees[~df_fees["Département"].str.contains("TOTAL")]
    df_fees = df_fees[df_fees["Effectifs"] != 0]
   # df_fees = df_fees[~df_fees["Spécialité"].str.contains("TOTAL")]
    df_fees = df_fees.dropna()
    
    
    df_fees["Type"] = type_doctor
    
    df_fees["Département"] = df_fees["Département"].apply(lambda x : x.split("-")[0])
    df_fees["Département"] = df_fees["Département"].astype(str)
   
    return df_fees

In [ ]:
medical_fees = pd.concat(map(fees_data, sheet_names))
medical_fees

In [ ]:
df_concat = pd.merge(df_population, medical_fees, how="left", on=["Département"])
df_concat = df_concat.dropna()

df_concat["Proportion dépassement"] = df_concat["Dépassements"]/df_concat["Total des honoraires"]

df_concat

In [ ]:
df_concat['Depassement ratio'] = df_concat['Dépassements'] / df_concat['Effectifs']
df_concat['Doctor density'] = df_concat['Effectifs'] / df_concat['Total']

In [ ]:
# Plotting results
plt.figure(figsize=(10, 6))
plt.plot(df_concat['Doctor density'], df_concat['Depassement ratio'],
         linestyle='', marker='o', color='b')
plt.xlabel('Doctor density per department')
plt.ylabel('Depassement ratio (Euros per doctor)')
plt.title("Relation between 'depassement' ratio and doctor density per department")
plt.show()